#### v0 30/05/2022
#### SCIENCE TEAM
#### THE NATURE CONSERVANCY


### JUPYTER NOTEBOOK TO AUTOMATIZE FOLDER PATH PROCESSING

##### TNC SCIENCE TEAM EFFORT TO SCRIPT DATA PROCESSING


In [1]:
#ref 1 https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/select-layer-by-location.htm
# ref 2 https://gis.stackexchange.com/questions/82250/how-to-get-sum-of-table-column-values-into-python-variable
#for learning/knowledge sharing purpose, we won't use functions here

import os
import pandas as pd
import glob

#path to hydrography folder to get ibge code list
hidrografia_folder =r"C:\Users\guilherme.fronza\Downloads\py_data\Hidrografia_rios_simples"

included_extensions = ['shp']
#select shapefiles to filenames variable
filenames = [fn for fn in os.listdir(hidrografia_folder)
              if any(fn.endswith(ext) for ext in included_extensions)]

#create empty list
cod_ibge_list = []

#loop through filenames and select cod ibge list
for filename in filenames:
    temp = filename.split("_")[1]
    cod_ibge_list.append(temp)
cod_ibge_list

['1500107', '1500131', '1500206']

In [2]:

#entire data folder (protected areas and hydrography layers)
data_folder = r"C:\Users\guilherme.fronza\Downloads\py_data"

#create empty lists to append shp paths
app_shp_list = []
hidrografia_shp_list = []

#nested loopings to walk through folders and subfolders to find rios_simples and app_uso shapefiles considering previous cod_ibge list.
for cod_ibge in cod_ibge_list:
    print(cod_ibge)
    for path, subdirs, files in os.walk(data_folder):
        for name in files:
            if name == (str('PA_' + cod_ibge) + '_RIOS_SIMPLES.shp'):
                #print('condicao 1')
                hidrografia_shp_list.append(os.path.join(path,name))
            elif name == (str('PA_' + cod_ibge) + '_APP_USO.shp'):
                #print('condicao 2')
                app_shp_list.append(os.path.join(path,name))
            else:
                None               

1500107
1500131
1500206


In [3]:
#print hydrography list
hidrografia_shp_list

['C:\\Users\\guilherme.fronza\\Downloads\\py_data\\Hidrografia_rios_simples\\PA_1500107_RIOS_SIMPLES.shp', 'C:\\Users\\guilherme.fronza\\Downloads\\py_data\\Hidrografia_rios_simples\\PA_1500131_RIOS_SIMPLES.shp', 'C:\\Users\\guilherme.fronza\\Downloads\\py_data\\Hidrografia_rios_simples\\PA_1500206_RIOS_SIMPLES.shp']

In [4]:
#print protected areas list
app_shp_list

['C:\\Users\\guilherme.fronza\\Downloads\\py_data\\APP_USO\\PA_1500107_APP_USO.shp', 'C:\\Users\\guilherme.fronza\\Downloads\\py_data\\APP_USO\\PA_1500131_APP_USO.shp', 'C:\\Users\\guilherme.fronza\\Downloads\\py_data\\APP_USO\\PA_1500206_APP_USO.shp']

In [5]:
#create empty list to append results (temporary)
result_list = []

#create list columns names
columns = ['COD IBGE', 'COMP_KM_SUM']

#nested looping to iterate over cod_ibge, protected areas and hydrography layers and process the data.
for hidrografia, app, cod_ibge in zip(hidrografia_shp_list, app_shp_list, cod_ibge_list):
    #a lot of prints (you can remove it)
    print(cod_ibge)
    print(hidrografia)
    print(app)
   
    #create the degraded rivers layer with select layer by location tool
    rios_degrad = arcpy.SelectLayerByLocation_management(hidrografia, "WITHIN_A_DISTANCE", app, "10 Meters", 
                                       "NEW_SELECTION", "NOT_INVERT")
    #initialize summed_total variable 
    summed_total = 0
    #loop through degraded river layer to sum degraded river lenghts (in kilometers)
    with arcpy.da.SearchCursor(rios_degrad, "COMP_KM") as cursor:
        for row in cursor:
            summed_total = summed_total + row[0]
    print(summed_total)
    #append results in the list 
    result_list.append({
         columns[0]: cod_ibge,
         columns[1]: summed_total
     })
#transform the result to a dataframe 
df_cod_ibge_river_sum = pd.DataFrame(result_list)

1500107
C:\Users\guilherme.fronza\Downloads\py_data\Hidrografia_rios_simples\PA_1500107_RIOS_SIMPLES.shp
C:\Users\guilherme.fronza\Downloads\py_data\APP_USO\PA_1500107_APP_USO.shp
998.9933753209189
1500131
C:\Users\guilherme.fronza\Downloads\py_data\Hidrografia_rios_simples\PA_1500131_RIOS_SIMPLES.shp
C:\Users\guilherme.fronza\Downloads\py_data\APP_USO\PA_1500131_APP_USO.shp
881.3870239257812
1500206
C:\Users\guilherme.fronza\Downloads\py_data\Hidrografia_rios_simples\PA_1500206_RIOS_SIMPLES.shp
C:\Users\guilherme.fronza\Downloads\py_data\APP_USO\PA_1500206_APP_USO.shp
2398.246295039309


In [6]:
#check the final result
df_cod_ibge_river_sum

,COD IBGE,COMP_KM_SUM
0,1500107,998.993375
1,1500131,881.387024
2,1500206,2398.246295


In [7]:
#export the final result to Excel file
df_cod_ibge_river_sum.to_excel("df_cod_ibge_river_sum.xlsx")

In [9]:
#check the path
#you can change it, try to :D 
current_path = os.getcwd()
print(current_path)

C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02e_dados_projeto
